In [29]:
import numpy as np
import re

In [30]:
def termIncidenceMatrix(l,n):
    docs = []
    terms = set()

    for i in l:
        file = open(i,'r').read().lower()
        content = re.sub("[^a-z0-9]"," ",file).split()
        terms.update(content)
        docs.append(set(content))

    mat = np.zeros((len(terms),n),dtype="bool")
    terms = sorted(terms)
    for i in range(len(terms)):
        for j in range(n):
            if terms[i] in docs[j]:
                mat[i][j] = 1
                
            else:
                mat[i][j] = 0
    return mat,terms

In [31]:
def precedence(op):
    if op == 'NOT':
        return 3
    elif op == 'AND':
        return 2
    elif op == 'OR':
        return 1
    else:
        return 0
    
def postfix(l):
    pf = []
    stack = []
    opList = ['AND','OR','NOT']
    for i in l:
        if i in opList:
            if stack == []:
                stack.append(i)
            else:
                while(stack != [] and precedence(i) <= precedence(stack[-1])):
                    pf.append(stack.pop())
                stack.append(i)
        elif i == '(':
            stack.append(i)
        elif i == ')':
            while(stack[-1] != '('):
                pf.append(stack.pop())
            stack.pop()
        else:
            pf.append(i)
    while(stack != []):
        pf.append(stack.pop())
    return pf

def queryEval(query,d):
    pf = postfix(query)
    stack = []
    opList = ['AND','OR','NOT']
    for i in pf:
        if i in opList:
            if i == 'NOT':
                op = stack.pop()
                if isinstance(op,str):
                    r = ~d[op]
                else:
                    r = ~op
            elif i == 'AND':
                op1 = stack.pop()
                op2 = stack.pop()
                if isinstance(op1,str):
                    op1 = d[op1]
                if isinstance(op2,str):
                    op2 = d[op2]
                r = op1&op2
            else:
                op1 = stack.pop()
                op2 = stack.pop()
                if isinstance(op1,str):
                    op1 = d[op1]
                if isinstance(op2,str):
                    op2 = d[op2]
                r = op1|op2
            stack.append(r)
        else:
            stack.append(i)
    return stack.pop()

In [33]:
n = 4
l = np.array(["doc1.txt","doc2.txt","doc3.txt","doc4.txt"])
# n = int(input("Enter the no. of documents : "))
# l = []
# for i in range(n):
#     l.append(input("Enter the doc - "+str(i+1)+" name : "))
# l = np.array(l)
TIM,terms = termIncidenceMatrix(l,n)
d = {}
for i in range(len(terms)):
    d[terms[i]] = TIM[i]
rules = """
The term insurance matrix have been created.
Rules for entering the query:
 1.There should be a single space between operand and operator
 2.There should be space before And after '(' , ')'
Enter the query:
"""
# query = input(rules).split()
resultDoc = queryEval("drug AND approach",d)
print(l[resultDoc])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices